In [1]:
import os, json

import pandas as pd
from langchain_openai import AzureChatOpenAI
from datetime import datetime

In [2]:
df_temp =pd.read_csv("df_claims_with_notes.csv")

In [3]:
# df_temp

In [4]:
df_claims = pd.read_csv("df_claims_with_notes.csv")

df_claims = df_claims.drop(columns=["is_valid_claim", "document","document_path"])
# df_claims = df_claims.drop(columns=["document","document_path"])
df_claim_and_rules = df_claims[["claim_type","covered_rules"]]

In [5]:
creds_path = r"C:\Pranav\azure_openai_creds.json"

with open(creds_path, "r") as f:
    creds = json.load(f)

llm = AzureChatOpenAI(
    azure_endpoint=creds["azure_endpoint"],
    api_key=creds["OPENAI_API_KEY"],
    api_version=creds["OPENAI_API_VERSION"],
    deployment_name=creds["deployment_name"]
)

In [6]:
# creds

In [7]:
df_claims["claim_type"].unique()

array(['Auto Insurance (Collision Coverage)',
       'Health Insurance (Major Medical Plan)',
       'Auto Insurance (Comprehensive Coverage)',
       'Homeowners Insurance (HO-3 Policy)', 'Travel Insurance',
       'Commercial Property Insurance', 'Workers’ Compensation Insurance',
       'Term Life Insurance'], dtype=object)

In [8]:
# for ind, row in df_claims.iterrows():

#     print(row["claim_type"], "\n")
#     # print(eval(row["covered_rules"]).keys(), "\n")
#     print(row["covered_rules"], "\n")
#     print(row["notes"], "\n")
#     # notes

In [9]:
df_claim_and_rules = df_claim_and_rules.drop_duplicates()

In [10]:
df_claims.head()

name policy_id policy_start  policy_end  \
0               Valid Vehicle  VHC-1001   2025-01-01  2025-12-31   
1               Valid Medical  HLT-2001   2025-01-01  2025-12-31   
2        Stolen Vehicle Claim  VHC-1002   2025-01-01  2025-12-31   
3          Water Damage Claim  HOM-3002   2025-01-01  2025-12-31   
4  Outpatient Procedure Claim  HLT-2002   2025-01-01  2025-12-31   

                                claim_type accident_date  claim_amount  \
0      Auto Insurance (Collision Coverage)    2025-06-20          5000   
1    Health Insurance (Major Medical Plan)    2025-05-15         12000   
2  Auto Insurance (Comprehensive Coverage)    2025-06-05         25000   
3       Homeowners Insurance (HO-3 Policy)    2025-03-12          6000   
4    Health Insurance (Major Medical Plan)    2025-06-10          3000   

                                       covered_rules  \
0  {'document_verification_agent': ['police_repor...   
1  {'document_verification_agent': ['submitted_do...   
2  {'document_verification_agent': ['police_repor...   
3  {'document_verification_agent': ['proof_of_los...   
4  {'document_verification_agent': ['submitted_do...   

                                               notes  
0  ['document_verification_agent - police_report_...  
1  ['document_verification_agent - submitted_docu...  
2  ['document_verification_agent - police_report_...  
3  ['document_verification_agent - proof_of_loss_...  
4  ['document_verification_agent - submitted_docu...

In [11]:
for ind, row in df_claims.iterrows():

    print(row["notes"])
    break

['document_verification_agent - police_report_validity: The police report is complete and legible, as inferred from the valid claim status.', 'document_verification_agent - photos_authenticity: Photographic evidence is recent and consistent, as inferred from the valid claim status.', 'document_verification_agent - repair_estimate_validity: Repair estimates are valid, as inferred from the valid claim status.', 'document_verification_agent - proof_of_insurance_validity: Proof of insurance is valid, as inferred from the valid claim status.', 'fraud_checker_agent - damage_consistency: Damages align with evidence, as inferred from the valid claim status.', 'fraud_checker_agent - repair_estimate_inconsistencies: No inconsistencies in repair estimates, as inferred from the valid claim status.', 'eligibility_checker_agent - driver_eligibility: The driver is authorized and covered by the policy, as inferred from the valid claim status.', 'fraud_checker_agent - duplicate_claim_prevention: No pri

In [12]:
# df_claim_and_rules.head(3)

In [13]:
df_claim_and_rules["covered_rules_formatted"] = None

In [14]:
for ind, row in df_claim_and_rules.iterrows():

    # print(row["claim_type"], "\n")
    covered_rules = eval(row["covered_rules"])
    # print(covered_rules, "\n")
    # print({i: {j[:j.index(":")] : j[j.index(":") + 2:]} for i, v in covered_rules.items() for j in v})
    dct = dict()

    for i, v in covered_rules.items():
        # print(v)
        # print(i)
        l1 = []
        for j in v:
            # print({j[:j.index(":")] : j[j.index(":") + 2:]})
            l1.append({j[:j.index(":")] : j[j.index(":") + 2:]})
        
        # print(l1)
        dct[i] = l1

    # print(dct)
    df_claim_and_rules.at[ind, "covered_rules_formatted"] = dct
    # break

In [15]:
# for ind, row in df_claim_and_rules.iterrows():

#     print(row["claim_type"], "\n")
#     covered_rules_formatted = row["covered_rules_formatted"]
#     # print(covered_rules_formatted, "\n")
#     for i, j in covered_rules_formatted.items():
#         print(i)
#         print(j)
#     # break

In [16]:
df_claim_and_rules = df_claim_and_rules.drop(columns="covered_rules")

In [17]:
df_claims = df_claims.drop(columns="covered_rules")

In [18]:
df_claims.head(2)

name policy_id policy_start  policy_end  \
0  Valid Vehicle  VHC-1001   2025-01-01  2025-12-31   
1  Valid Medical  HLT-2001   2025-01-01  2025-12-31   

                              claim_type accident_date  claim_amount  \
0    Auto Insurance (Collision Coverage)    2025-06-20          5000   
1  Health Insurance (Major Medical Plan)    2025-05-15         12000   

                                               notes  
0  ['document_verification_agent - police_report_...  
1  ['document_verification_agent - submitted_docu...

In [19]:
for ind, row in df_claims.iterrows():

    print(row.to_dict())
    break

{'name': 'Valid Vehicle', 'policy_id': 'VHC-1001', 'policy_start': '2025-01-01', 'policy_end': '2025-12-31', 'claim_type': 'Auto Insurance (Collision Coverage)', 'accident_date': '2025-06-20', 'claim_amount': 5000, 'notes': "['document_verification_agent - police_report_validity: The police report is complete and legible, as inferred from the valid claim status.', 'document_verification_agent - photos_authenticity: Photographic evidence is recent and consistent, as inferred from the valid claim status.', 'document_verification_agent - repair_estimate_validity: Repair estimates are valid, as inferred from the valid claim status.', 'document_verification_agent - proof_of_insurance_validity: Proof of insurance is valid, as inferred from the valid claim status.', 'fraud_checker_agent - damage_consistency: Damages align with evidence, as inferred from the valid claim status.', 'fraud_checker_agent - repair_estimate_inconsistencies: No inconsistencies in repair estimates, as inferred from th

In [20]:
df_claim_and_rules.head(2)

claim_type  \
0    Auto Insurance (Collision Coverage)   
1  Health Insurance (Major Medical Plan)   

                             covered_rules_formatted  
0  {'document_verification_agent': [{'police_repo...  
1  {'document_verification_agent': [{'submitted_d...

In [21]:
df_claim_and_rules["claim_type"].unique()

array(['Auto Insurance (Collision Coverage)',
       'Health Insurance (Major Medical Plan)',
       'Auto Insurance (Comprehensive Coverage)',
       'Homeowners Insurance (HO-3 Policy)', 'Travel Insurance',
       'Commercial Property Insurance', 'Workers’ Compensation Insurance',
       'Term Life Insurance'], dtype=object)

In [22]:
# for ind, row in df_claim_and_rules.iterrows():
#     # print(row["claim_type"])
#     # print(row["covered_rules_formatted"])
#     print(row.to_dict())
#     print()

In [23]:
# df_claims.columns

In [24]:
# df_claim_and_rules

In [25]:
# def validate_claim(claim_row, df_claim_and_rules, llm):
#     claim_type = claim_row['claim_type']
#     policy_id = claim_row["policy_id"]
#     rules_row = df_claim_and_rules[df_claim_and_rules['claim_type'] == claim_type]
#     if rules_row.empty:
#         return {"error": "No rules found for claim type"}
#     rules = rules_row.iloc[0]['covered_rules_formatted']

#     prompt = f"""
#     You are an AI assistant tasked with validating an insurance claim based on provided claim details and predefined rules. You will receive:
#     1. A claim record with fields: `name`, `policy_id`, `policy_start`, `policy_end`, `claim_type`, `accident_date`, `claim_amount`, and `notes` (a list of strings indicating rule validations or inferences).
#     2. A set of rules for the claim type, organized by three agents: `document_verification_agent`, `eligibility_checker_agent`, and `fraud_checker_agent`. Each agent has specific rules with conditions that must be met.

#     Your task is to:
#     - Validate the claim record against the rules for the corresponding `claim_type`.
#     - For each agent (`document_verification_agent`, `eligibility_checker_agent`, `fraud_checker_agent`):
#       - Check the `notes` for explicit rule validations (e.g., "document_verification_agent - police_report_validity: The police report is complete and legible").
#       - For rules not explicitly mentioned in `notes`, infer compliance based on the claim fields (e.g., check `accident_date` against `policy_start` and `policy_end` for `incident_date_verification`).
#       - If any rule for an agent fails, the agent's status is "failed". All rules must pass for the agent to have a "passed" status.
#     - Assign a confidence score (between 0.0 and 1.0) based on the strength of the evidence or inference.
#     - Generate a JSON output with one entry per agent, including:
#       - `timestamp`: Use "2025-06-26T16:12:00".
#       - `claim_id`: Constructed as `CLM_{{policy_id}}`.
#       - `step`: The agent name (e.g., `document_verification_agent`).
#       - `status`: Either "passed" or "failed".
#       - `reason`: A concise explanation of the validation result.
#       - `confidence`: A float between 0.0 and 1.0 reflecting confidence in the result.
#     - Output STRICTLY the JSON result, with no additional text, explanations, or markdown.

#     **Input Data:**

#     Claim Record:  
#     {json.dumps(claim_row.to_dict())}

#     Rules for Claim Type '{claim_type}':  
#     {json.dumps(rules)}

#     **Instructions:**
#     1. For each agent:
#        - Check the `notes` for explicit rule validations.
#        - For rules not in `notes`, infer compliance based on fields. For example:
#          - For `incident_date_verification`, check if `accident_date` falls between `policy_start` and `policy_end`.
#          - For `policy_active_status`, assume premiums are paid unless contradicted.
#          - For rules requiring external data (e.g., `incident_veracity`), use notes or assume partial compliance with lower confidence if no note exists.
#        - If any rule for an agent fails, the agent's status is "failed". All rules must pass for the agent to have a "passed" status.
#     2. Generate a confidence score:
#        - Use 0.9 for rules explicitly validated in notes.
#        - Use 0.6–0.8 for inferred validations based on fields.
#        - Use 0.4–0.5 for rules requiring external data with no explicit note.
#     3. Output the result in the following JSON format, with no additional text:

#     [
#       {{
#         "timestamp": "2025-06-26T16:12:00",
#         "claim_id": "CLM_{{policy_id}}",
#         "step": "document_verification_agent",
#         "status": "<passed/failed>",
#         "reason": "<explanation>",
#         "confidence": <float>
#       }},
#       {{
#         "timestamp": "2025-06-26T16:12:00",
#         "claim_id": "CLM_{{policy_id}}",
#         "step": "eligibility_checker_agent",
#         "status": "<passed/failed>",
#         "reason": "<explanation>",
#         "confidence": <float>
#       }},
#       {{
#         "timestamp": "2025-06-26T16:12:00",
#         "claim_id": "CLM_{{policy_id}}",
#         "step": "fraud_checker_agent",
#         "status": "<passed/failed>",
#         "reason": "<explanation>",
#         "confidence": <float>
#       }}
#     ]

#     **Output:**

#     Generate the JSON output based on the validation of the claim record against the rules. Use the timestamp "2025-06-26T16:12:00". If any information is missing or ambiguous, make reasonable assumptions and reflect uncertainty in the confidence score. Output only the JSON, with no additional text or markdown.
#     """

#     try:
#         response = llm.invoke(prompt)
#         result = json.loads(response.content)
#         return result
#     except Exception as e:
#         return {"error": f"LLM invocation failed: {str(e)}"}
    
# df_claims["agents_output_temp"] = None

# for ind, row in df_claims.iterrows():

#     # print(ind)
#     # print(type(row.to_dict()))
#     claim_row = row#.to_dict()
#     output = validate_claim(claim_row, df_claim_and_rules, llm)
#     # print(type(output))
#     df_claims.at[ind, "agents_output_temp"] = output

In [26]:
# for ind, row in df_claims.iterrows():

#     if ind == 2:
        
#         # print(type(row.to_dict()))
#         claim_row = row#.to_dict()
#         output = validate_claim(claim_row, df_claim_and_rules, llm)
#         # print(type(output))
#         print(output)

In [27]:
df_claims.head(2)

name policy_id policy_start  policy_end  \
0  Valid Vehicle  VHC-1001   2025-01-01  2025-12-31   
1  Valid Medical  HLT-2001   2025-01-01  2025-12-31   

                              claim_type accident_date  claim_amount  \
0    Auto Insurance (Collision Coverage)    2025-06-20          5000   
1  Health Insurance (Major Medical Plan)    2025-05-15         12000   

                                               notes  
0  ['document_verification_agent - police_report_...  
1  ['document_verification_agent - submitted_docu...

In [28]:
df_claims.head(2)

name policy_id policy_start  policy_end  \
0  Valid Vehicle  VHC-1001   2025-01-01  2025-12-31   
1  Valid Medical  HLT-2001   2025-01-01  2025-12-31   

                              claim_type accident_date  claim_amount  \
0    Auto Insurance (Collision Coverage)    2025-06-20          5000   
1  Health Insurance (Major Medical Plan)    2025-05-15         12000   

                                               notes  
0  ['document_verification_agent - police_report_...  
1  ['document_verification_agent - submitted_docu...

In [29]:
# df_claims

In [30]:
# df_claims.to_csv("df_claims_temperory_output.csv", index=False)
df_claims = pd.read_csv("df_claims_temperory_output.csv")

In [31]:
# for ind, row in df_claims.iterrows():

#     print(row["name"], "\n")
#     print(row["claim_type"], "\n")
#     print(row["agents_output_temp"], "\n")
#     # print([{k:v} for i in row["agents_output_temp"] for k, v in i.items() if k in ["status","reason"]], "\n")

In [32]:
df_claims.head(2)

name policy_id policy_start  policy_end  \
0  Valid Vehicle  VHC-1001   2025-01-01  2025-12-31   
1  Valid Medical  HLT-2001   2025-01-01  2025-12-31   

                              claim_type accident_date  claim_amount  \
0    Auto Insurance (Collision Coverage)    2025-06-20          5000   
1  Health Insurance (Major Medical Plan)    2025-05-15         12000   

                                               notes  \
0  ['document_verification_agent - police_report_...   
1  ['document_verification_agent - submitted_docu...   

                                  agents_output_temp  
0  [{'timestamp': '2025-06-26T16:12:00', 'claim_i...  
1  [{'timestamp': '2025-06-26T16:12:00', 'claim_i...

In [33]:
df_claim_and_rules#.head(2)

claim_type  \
0      Auto Insurance (Collision Coverage)   
1    Health Insurance (Major Medical Plan)   
2  Auto Insurance (Comprehensive Coverage)   
3       Homeowners Insurance (HO-3 Policy)   
5                         Travel Insurance   
6            Commercial Property Insurance   
8          Workers’ Compensation Insurance   
9                      Term Life Insurance   

                             covered_rules_formatted  
0  {'document_verification_agent': [{'police_repo...  
1  {'document_verification_agent': [{'submitted_d...  
2  {'document_verification_agent': [{'police_repo...  
3  {'document_verification_agent': [{'proof_of_lo...  
5  {'document_verification_agent': [{'required_do...  
6  {'document_verification_agent': [{'submitted_d...  
8  {'document_verification_agent': [{'incident_re...  
9  {'document_verification_agent': [{'death_certi...

In [34]:
for ind, row in df_claim_and_rules.iterrows():

    # print(row["claim_type"])
    # print(row["covered_rules_formatted"])
    for k, v in row["covered_rules_formatted"].items():
        print(k)
        print(v)
    break

document_verification_agent
[{'police_report_validity': 'The police report must be complete and legible.'}, {'photos_authenticity': 'Photographic evidence of the damage must be recent and consistent with the reported incident.'}, {'repair_estimate_validity': 'Repair estimates must be provided with details from recognized service providers.'}, {'proof_of_insurance_validity': "The submitted proof of insurance must match the policyholder's information and be current."}]
eligibility_checker_agent
[{'policy_active_status': 'The policy must be active with all premiums paid up to date at the time of the incident.'}, {'coverage_compliance': 'The damages claimed must fall under collision coverage as outlined in the policy terms.'}, {'incident_date_verification': 'The incident must occur within the covered policy period.'}, {'driver_eligibility': 'The driver involved in the incident must be authorized and covered by the policy.'}]
fraud_checker_agent
[{'damage_consistency': 'The claimed damages 

In [35]:
# def generate_prompts_column(df_claim_and_rules):
#     def create_agent_prompt(agent_name, rules, claim_type):
#         return f"""
# You are an AI assistant tasked with validating an insurance claim for the {agent_name} based on provided claim details and predefined rules. You will receive:
# 1. A claim record with fields: `name`, `policy_id`, `policy_start`, `policy_end`, `claim_type`, `accident_date`, `claim_amount`, and `notes` (a list of strings indicating rule validations or inferences).
# 2. A set of rules specific to the {agent_name} for the claim type '{claim_type}'.

# Your task is to:
# - Validate the claim record against the provided rules for the {agent_name}.
# - Check the `notes` for explicit rule validations (e.g., "{agent_name} - rule_name: description").
# - For rules not explicitly mentioned in `notes`, infer compliance based on the claim fields (e.g., check `accident_date` against `policy_start` and `policy_end` for `incident_date_verification`).
# - If any rule fails, the agent's status is "failed". All rules must pass for the status to be "passed".
# - Assign a confidence score (between 0.0 and 1.0) based on the strength of the evidence or inference:
#   - Use 0.9 for rules explicitly validated in notes.
#   - Use 0.6–0.8 for inferred validations based on fields.
#   - Use 0.4–0.5 for rules requiring external data with no explicit note.
# - Output STRICTLY a JSON result with a single entry for the {agent_name}, including:
#   - `timestamp`: Use "2025-06-26T16:45:00".
#   - `claim_id`: Constructed as `CLM_{{policy_id}}`.
#   - `step`: The agent name, "{agent_name}".
#   - `status`: Either "passed" or "failed".
#   - `reason`: A concise explanation of the validation result.
#   - `confidence`: A float between 0.0 and 1.0 reflecting confidence in the result.

# **Input Data:**

# Claim Record:  
# {{claim_record}}

# Rules for {agent_name} (Claim Type '{claim_type}'):  
# {json.dumps(rules)}

# **Instructions:**
# 1. For the {agent_name}:
#    - Check the `notes` for explicit rule validations.
#    - For rules not in `notes`, infer compliance based on fields. For example:
#      - For `incident_date_verification`, check if `accident_date` falls between `policy_start` and `policy_end`.
#      - For `policy_active_status`, assume premiums are paid unless contradicted.
#      - For rules requiring external data (e.g., `incident_veracity`), use notes or assume partial compliance with lower confidence if no note exists.
#    - If any rule fails, the status is "failed". All rules must pass for "passed".
# 2. Output the result in the following JSON format, with no additional text:

# [
#   {{
#     "timestamp": "2025-06-26T16:45:00",
#     "claim_id": "CLM_{{policy_id}}",
#     "step": "{agent_name}",
#     "status": "<passed/failed>",
#     "reason": "<explanation>",
#     "confidence": <float>
#   }}
# ]
# """

#     prompts_list = []
#     for _, row in df_claim_and_rules.iterrows():
#         claim_type = row['claim_type']
#         rules = row['covered_rules_formatted']
#         agent_prompts = []
#         for agent in ['document_verification_agent', 'eligibility_checker_agent', 'fraud_checker_agent']:
#             agent_rules = rules.get(agent, [])
#             prompt = create_agent_prompt(agent, agent_rules, claim_type)
#             agent_prompts.append(prompt)
#         prompts_list.append(agent_prompts)
    
#     df_claim_and_rules['prompts'] = prompts_list
#     return df_claim_and_rules

# df_claim_and_rules = generate_prompts_column(df_claim_and_rules)

In [36]:
# for ind, row in df_claim_and_rules.iterrows():

#     print(row["claim_type"], "\n")
#     # print(, "\n")
#     for i, v in row["covered_rules_formatted"].items():
#         print(i)
#         print(v)
#     # print(row["prompts"], "\n")
#     for i in row["prompts"]:
#         print(i)

In [37]:
def generate_prompts_column(df_claim_and_rules):
    def create_agent_prompt(agent_name, rules, claim_type, rule_name=None):
        rule_desc = f"Rule '{rule_name}'" if rule_name else "Provided Rules"
        return f"""
You are an AI assistant tasked with validating an insurance claim for the {agent_name} based on provided claim details and predefined rules. You will receive:
1. A claim record with fields: `name`, `policy_id`, `policy_start`, `policy_end`, `claim_type`, `accident_date`, `claim_amount`, and `notes` (a list of strings indicating rule validations or inferences).
2. {rule_desc} for the {agent_name} for the claim type '{claim_type}'.

Your task is to:
- Validate the claim record against the provided rules for the {agent_name}.
- Check the `notes` for explicit rule validations (e.g., "{agent_name}{f' - {rule_name}' if rule_name else ''}: description").
- For rules not explicitly mentioned in `notes`, infer compliance based on the claim fields (e.g., check `accident_date` against `policy_start` and `policy_end` for `incident_date_verification`).
- If any rule fails, the agent's status is "failed". All rules must pass for the status to be "passed".
- Assign a confidence score (between 0.0 and 1.0) based on the strength of the evidence or inference:
  - Use 0.9 for rules explicitly validated in notes.
  - Use 0.6–0.8 for inferred validations based on fields.
  - Use 0.4–0.5 for rules requiring external data with no explicit note.
- Output STRICTLY a JSON result with a single entry for the {agent_name}, including:
  - `timestamp`: Use "2025-06-26T16:45:00".
  - `claim_id`: Constructed as `CLM_{{policy_id}}`.
  - `step`: The agent name, "{agent_name}"{f" (Rule: {rule_name})" if rule_name else ""}.
  - `status`: Either "passed" or "failed".
  - `reason`: A concise explanation of the validation result.
  - `confidence`: A float between 0.0 and 1.0 reflecting confidence in the result.

**Input Data:**

Claim Record:  
{{claim_record}}

Rules for {agent_name} (Claim Type '{claim_type}'){f" - Rule '{rule_name}'" if rule_name else ""}:  
{json.dumps(rules)}

**Instructions:**
1. For the {agent_name}:
   - Check the `notes` for explicit rule validations.
   - For rules not in `notes`, infer compliance based on fields. For example:
     - For `incident_date_verification`, check if `accident_date` falls between `policy_start` and `policy_end`.
     - For `policy_active_status`, assume premiums are paid unless contradicted.
     - For rules requiring external data (e.g., `incident_veracity`), use notes or assume partial compliance with lower confidence if no note exists.
   - If any rule fails, the status is "failed". All rules must pass for "passed".
2. Output the result in the following JSON format, with no additional text:

[
  {{
    "timestamp": "2025-06-26T16:45:00",
    "claim_id": "CLM_{{policy_id}}",
    "step": "{agent_name}{f' (Rule: {rule_name})' if rule_name else ''}",
    "status": "<passed/failed>",
    "reason": "<explanation>",
    "confidence": <float>
  }}
]
"""

    prompts_list = []
    for _, row in df_claim_and_rules.iterrows():
        claim_type = row['claim_type']
        rules = row['covered_rules_formatted']
        agent_prompts = []
        
        # Create one prompt for document_verification_agent
        prompt = create_agent_prompt(
            'document_verification_agent',
            rules.get('document_verification_agent', []),
            claim_type
        )
        agent_prompts.append({
            "agent": "document_verification_agent",
            "rule": "all",
            "prompt": prompt
        })
        
        # Create one prompt for eligibility_checker_agent
        prompt = create_agent_prompt(
            'eligibility_checker_agent',
            rules.get('eligibility_checker_agent', []),
            claim_type
        )
        agent_prompts.append({
            "agent": "eligibility_checker_agent",
            "rule": "all",
            "prompt": prompt
        })
        
        # Create one prompt per rule for fraud_checker_agent
        fraud_rules = rules.get('fraud_checker_agent', [])
        for rule in fraud_rules:
            rule_name = list(rule.keys())[0]
            rule_description = list(rule.values())[0]
            prompt = create_agent_prompt(
                'fraud_checker_agent',
                [{rule_name: rule_description}],
                claim_type,
                rule_name=rule_name
            )
            agent_prompts.append({
                "agent": "fraud_checker_agent",
                "rule": rule_name,
                "prompt": prompt
            })
        
        prompts_list.append(agent_prompts)
    
    df_claim_and_rules['prompts'] = prompts_list
    return df_claim_and_rules

In [38]:
df_claim_and_rules.head(2)

claim_type  \
0    Auto Insurance (Collision Coverage)   
1  Health Insurance (Major Medical Plan)   

                             covered_rules_formatted  
0  {'document_verification_agent': [{'police_repo...  
1  {'document_verification_agent': [{'submitted_d...

In [39]:
# df_claim_and_rules = df_claim_and_rules.drop(columns="prompts")

In [40]:
df_claim_and_rules = generate_prompts_column(df_claim_and_rules)

In [41]:
# for _, row in df_claim_and_rules.iterrows():

#     rules = row['covered_rules_formatted']
#     fraud_rules = rules.get('fraud_checker_agent', [])
#     for rule in fraud_rules:
#         print(rule)

In [42]:
df_claim_and_rules.head(2)

claim_type  \
0    Auto Insurance (Collision Coverage)   
1  Health Insurance (Major Medical Plan)   

                             covered_rules_formatted  \
0  {'document_verification_agent': [{'police_repo...   
1  {'document_verification_agent': [{'submitted_d...   

                                             prompts  
0  [{'agent': 'document_verification_agent', 'rul...  
1  [{'agent': 'document_verification_agent', 'rul...

In [43]:
for ind, row in df_claim_and_rules.iterrows():
    
    # print(row["covered_rules_formatted"])
    if ind == 6:
        for i in row["prompts"]:
            print(i["prompt"], "\n")

        break


You are an AI assistant tasked with validating an insurance claim for the document_verification_agent based on provided claim details and predefined rules. You will receive:
1. A claim record with fields: `name`, `policy_id`, `policy_start`, `policy_end`, `claim_type`, `accident_date`, `claim_amount`, and `notes` (a list of strings indicating rule validations or inferences).
2. Provided Rules for the document_verification_agent for the claim type 'Commercial Property Insurance'.

Your task is to:
- Validate the claim record against the provided rules for the document_verification_agent.
- Check the `notes` for explicit rule validations (e.g., "document_verification_agent: description").
- For rules not explicitly mentioned in `notes`, infer compliance based on the claim fields (e.g., check `accident_date` against `policy_start` and `policy_end` for `incident_date_verification`).
- If any rule fails, the agent's status is "failed". All rules must pass for the status to be "passed".
- A

In [44]:
# for ind, row in df_claim_and_rules.iterrows():

#     # print(row["covered_rules_formatted"], "\n")
#     # print(row["prompts"], "\n")
#     for k, v in row["covered_rules_formatted"].items():
#         print(k, "\n")
#         print(v, "\n")
    
#     for i in row["prompts"]:
#         print(i, "\n")

#     break

In [45]:
# df_claim_and_rules["covered_rules_formatted"][0]

In [46]:
# df_claims.head(2)

In [47]:
# df_claim_and_rules.to_csv("df_claim_and_rules_with_prompts.csv", index=False)